## Aim of This Notebook 

The aim of this notebook is to create Latitude and Longitude from n32 and e32 columns .

In [1]:
import pandas as pd
import psycopg2
import utm
import numpy as np
import sqlalchemy
import credential_temp as  creds 

## Connect to the Database

In [2]:
conn_string = "host=" + creds.PGHOST + " port=" + creds.PORT + " dbname=" + creds.PGDATABASE + " user=" + creds.PGUSER \
    + " password=" + creds.PGPASSWORD

In [3]:
conn = psycopg2.connect(conn_string)

## Read the Table from Database

In this step, we need to read the messstelle table which is already in our database to the pandas dataframe.

In [4]:
quary = """ select * from hygrisc.messstelle """

In [5]:
df = pd.read_sql(quary, conn,)

In [6]:
df 

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gwhorizont_id,gwhorizont,...,eigentuemer,betreiber,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm,geom
0,6979,32502539,UWB-Ddorf 00204A,342735,5677483,1.0,oeffentlich,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E8640000000000003CEB1441000000C06AA8...
1,7068,32503428,UWB-Ddorf 00282,350898,5671796,1.0,oeffentlich,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E864000000000000C86A154100000000DDA2...
2,7111,32503799,UWB-Ddorf 00318,350579,5680395,1.0,privat,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E864000000000000CC651541000000C042AB...
3,7175,32504354,UWB-Ddorf 00382,342941,5682278,1.0,oeffentlich,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E86400000000000074EE14410000008019AD...
4,7178,32504380,UWB-Ddorf 00387,348127,5674011,1.0,privat,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E8640000000000007C3F1541000000C006A5...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71114,43118,288066315,Ramrath,337994,5663209,1.0,oeffentlich,05162028,5,Zwischenmittel,...,"RBW TGB, Garzweiler ...","RBW TGB, Garzweiler ...",216.0,210.0,50.0,NaN,NaN,NaN,NaN,0101000020E86400000000000028A11441000000407A9A...
71115,43531,288120012,Innenkippe Sï¿½d,331043,5655334,1.0,privat,05162008,20,"Anthropogene Bildungen (Aufschüttungen, Kippen)",...,"RBW TGB, Garzweiler ...","RBW TGB, Garzweiler ...",1800.0,300.0,50.0,NaN,NaN,NaN,NaN,0101000020E8640000000000008C34144100000080C992...
71116,50990,10446205,60DP002002,289533,5651548,NaN,privat,NL000881,9A,Unterer (Haupt-) Rotton,...,Prov. Limburg (NL) ...,Prov. Limburg (NL) ...,100.0,NaN,NaN,NaN,6591.0,-387.0,-487.0,0101000020E864000000000000F4AB114100000000178F...
71117,6872,32501432,UWB-Ddorf 00111Z,351514,5676929,1.0,privat,05111000,None,None,...,Stadt Düsseldorf ...,Stadt Düsseldorf ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E8640000000000006874154100000040E0A7...


## Convert e32 and n32 to lat and Long

First we need to check the utm function

In [7]:
# u = utm.from_latlon(467220, 5755927)
utm.to_latlon(467220, 5755927, 32, 'N')
# >>> print(u)


(51.95307898584043, 8.52300647049394)

As we can see in the above, our e32 and n32 have been converted to lat and long 

Now we need to create our function with utm and apply this function on the entire data frame. 

In [8]:
# function
def convert(lat, long):
    a = utm.to_latlon(lat, long, 32, 'N')
    return a

   

In [ ]:
df = df.drop(df[df.e32 == 50].index)

In [ ]:
filt = df['n32'] == 50
df[filt]

In [11]:
# apply function
df['lat'] = df.apply(lambda x: convert(x['e32'], x['n32'])[0], axis=1)
df['long'] = df.apply(lambda x: convert(x['e32'], x['n32'])[1], axis=1)


In [12]:
df

,sl_nr,messstelle_id,name,e32,n32,gw_stockwerk,grundstueck,gemeinde_id,gwhorizont_id,gwhorizont,...,filterlaenge_cm,sumpfrohrlaenge_cm,ausbaudurchmesser_mm,historischer_ruhe_wsp,einbaulaenge_cm,oberkante_filter_cm,unterkante_filter_cm,geom,lat,long
0,6979,32502539,UWB-Ddorf 00204A,342735,5677483,1.0,oeffentlich,05111000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E8640000000000003CEB1441000000C06AA8...,51.227035,6.747653
1,7068,32503428,UWB-Ddorf 00282,350898,5671796,1.0,oeffentlich,05111000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E864000000000000C86A154100000000DDA2...,51.178123,6.866837
2,7111,32503799,UWB-Ddorf 00318,350579,5680395,1.0,privat,05111000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E864000000000000CC651541000000C042AB...,51.255309,6.858694
3,7175,32504354,UWB-Ddorf 00382,342941,5682278,1.0,oeffentlich,05111000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E86400000000000074EE14410000008019AD...,51.270175,6.748497
4,7178,32504380,UWB-Ddorf 00387,348127,5674011,1.0,privat,05111000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E8640000000000007C3F1541000000C006A5...,51.197297,6.826287
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
71114,43118,288066315,Ramrath,337994,5663209,1.0,oeffentlich,05162028,5,Zwischenmittel,...,216.0,210.0,50.0,NaN,NaN,NaN,NaN,0101000020E86400000000000028A11441000000407A9A...,51.097459,6.686240
71115,43531,288120012,Innenkippe Sï¿½d,331043,5655334,1.0,privat,05162008,20,"Anthropogene Bildungen (Aufschüttungen, Kippen)",...,1800.0,300.0,50.0,NaN,NaN,NaN,NaN,0101000020E8640000000000008C34144100000080C992...,51.024700,6.590735
71116,50990,10446205,60DP002002,289533,5651548,NaN,privat,NL000881,9A,Unterer (Haupt-) Rotton,...,100.0,NaN,NaN,NaN,6591.0,-387.0,-487.0,0101000020E864000000000000F4AB114100000000178F...,50.977008,6.001794
71117,6872,32501432,UWB-Ddorf 00111Z,351514,5676929,1.0,privat,05111000,None,None,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0101000020E8640000000000006874154100000040E0A7...,51.224408,6.873520


## Connect to Database

In [14]:
postgresurl = creds.URL


In [16]:
engine = sqlalchemy.create_engine(postgresurl)

## Write to Database

In [17]:
df.to_sql(con=engine, name="messstelle_lat",schema="hygrisc", if_exists="append", index=False)

# Finish !
## Good Luck, You are doing well